# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 9:10AM,237626,18,HH-34411,Hush Hush,Released
1,May 19 2022 9:10AM,237625,18,HH-34409,Hush Hush,Released
2,May 19 2022 9:10AM,237625,18,HH-34410,Hush Hush,Released
3,May 19 2022 9:10AM,237625,18,HH-34412,Hush Hush,Released
4,May 19 2022 9:10AM,237625,18,HH-34413,Hush Hush,Released
5,May 19 2022 9:10AM,237625,18,HH-34414,Hush Hush,Released
6,May 19 2022 9:10AM,237625,18,HH-34415,Hush Hush,Released
7,May 19 2022 9:10AM,237625,18,HH-34416,Hush Hush,Released
8,May 19 2022 9:10AM,237625,18,HH-34417,Hush Hush,Released
9,May 19 2022 9:09AM,237627,15,PNC356230B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,237623,Released,1
57,237625,Released,8
58,237626,Released,1
59,237627,Released,17
60,237628,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237623,NaN,NaN,NaN,1.0
237625,NaN,NaN,NaN,8.0
237626,NaN,NaN,NaN,1.0
237627,NaN,NaN,NaN,17.0
237628,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237507,1.0,13.0,2.0,0.0
237513,0.0,0.0,0.0,1.0
237514,0.0,0.0,0.0,1.0
237529,0.0,0.0,0.0,2.0
237531,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
237507,1.0,13,2,0
237513,0.0,0,0,1
237514,0.0,0,0,1
237529,0.0,0,0,2
237531,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,237507,1.0,13,2,0
1,237513,0.0,0,0,1
2,237514,0.0,0,0,1
3,237529,0.0,0,0,2
4,237531,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,237507,1.0,13,2,
1,237513,0.0,,,1
2,237514,0.0,,,1
3,237529,0.0,,,2
4,237531,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 9:10AM,237626,18,Hush Hush
1,May 19 2022 9:10AM,237625,18,Hush Hush
9,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc."
26,May 19 2022 9:08AM,237628,10,ISDIN Corporation
27,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC"
28,May 19 2022 9:05AM,237622,10,ISDIN Corporation
71,May 19 2022 9:05AM,237619,15,"Alexso, Inc."
72,May 19 2022 9:04AM,237618,15,"Mizner Bioscience, LLC"
82,May 19 2022 9:04AM,237617,15,"Carwin Pharmaceutical Associates, LLC"
88,May 19 2022 8:59AM,237620,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,May 19 2022 9:10AM,237626,18,Hush Hush,0.0,,,1
1,May 19 2022 9:10AM,237625,18,Hush Hush,0.0,,,8
2,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc.",0.0,,,17
3,May 19 2022 9:08AM,237628,10,ISDIN Corporation,0.0,,,1
4,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC",0.0,,,1
5,May 19 2022 9:05AM,237622,10,ISDIN Corporation,0.0,,,43
6,May 19 2022 9:05AM,237619,15,"Alexso, Inc.",0.0,,,1
7,May 19 2022 9:04AM,237618,15,"Mizner Bioscience, LLC",0.0,,,10
8,May 19 2022 9:04AM,237617,15,"Carwin Pharmaceutical Associates, LLC",0.0,,,6
9,May 19 2022 8:59AM,237620,19,ACG North America LLC,0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 9:10AM,237626,18,Hush Hush,1,,
1,May 19 2022 9:10AM,237625,18,Hush Hush,8,,
2,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc.",17,,
3,May 19 2022 9:08AM,237628,10,ISDIN Corporation,1,,
4,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC",1,,
5,May 19 2022 9:05AM,237622,10,ISDIN Corporation,43,,
6,May 19 2022 9:05AM,237619,15,"Alexso, Inc.",1,,
7,May 19 2022 9:04AM,237618,15,"Mizner Bioscience, LLC",10,,
8,May 19 2022 9:04AM,237617,15,"Carwin Pharmaceutical Associates, LLC",6,,
9,May 19 2022 8:59AM,237620,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 9:10AM,237626,18,Hush Hush,1,,
1,May 19 2022 9:10AM,237625,18,Hush Hush,8,,
2,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc.",17,,
3,May 19 2022 9:08AM,237628,10,ISDIN Corporation,1,,
4,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 9:10AM,237626,18,Hush Hush,1.0,NaN,NaN
1,May 19 2022 9:10AM,237625,18,Hush Hush,8.0,NaN,NaN
2,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc.",17.0,NaN,NaN
3,May 19 2022 9:08AM,237628,10,ISDIN Corporation,1.0,NaN,NaN
4,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 9:10AM,237626,18,Hush Hush,1.0,0.0,0.0
1,May 19 2022 9:10AM,237625,18,Hush Hush,8.0,0.0,0.0
2,May 19 2022 9:09AM,237627,15,"Person & Covey, Inc.",17.0,0.0,0.0
3,May 19 2022 9:08AM,237628,10,ISDIN Corporation,1.0,0.0,0.0
4,May 19 2022 9:05AM,237623,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3801439,169.0,9.0,16.0
15,1663334,113.0,0.0,0.0
16,1425458,9.0,16.0,0.0
18,1663077,16.0,2.0,13.0
19,712812,31.0,14.0,0.0
20,1900502,105.0,12.0,0.0
21,475151,2.0,0.0,0.0
22,237571,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3801439,169.0,9.0,16.0
1,15,1663334,113.0,0.0,0.0
2,16,1425458,9.0,16.0,0.0
3,18,1663077,16.0,2.0,13.0
4,19,712812,31.0,14.0,0.0
5,20,1900502,105.0,12.0,0.0
6,21,475151,2.0,0.0,0.0
7,22,237571,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,169.0,9.0,16.0
1,15,113.0,0.0,0.0
2,16,9.0,16.0,0.0
3,18,16.0,2.0,13.0
4,19,31.0,14.0,0.0
5,20,105.0,12.0,0.0
6,21,2.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,169.0
1,15,Released,113.0
2,16,Released,9.0
3,18,Released,16.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,16.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,16.0,2.0,14.0,12.0,0.0,1.0
Released,169.0,113.0,9.0,16.0,31.0,105.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,16.0,2.0,14.0,12.0,0.0,1.0
2,Released,169.0,113.0,9.0,16.0,31.0,105.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,16.0,2.0,14.0,12.0,0.0,1.0
2,Released,169.0,113.0,9.0,16.0,31.0,105.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()